In [96]:
import yt
import os
import numpy as np
import matplotlib.pyplot as plt

valueString = 'Pressure' # which value we want to plot
# possible values are: ['Density', 'Energy', 'Momentum_0', 'Momentum_1', 'Pressure', 'Species_0', 'Species_1', 'SpeedOfSound', 'vfrac']

dataPath = "/home/christian/FVS_develop/FiniteVolumeSolver/build_Release_2d_EB/SEC_Plenum/"
plenum_number = 0
MASK_DATA_WITH_VFRAC = True
vfrac_cutoff = 1.0e-16

def loadData(plotfile, var):
  ds = yt.load(plotfile)
  ad = ds.all_data()
  current_time = ds.current_time
  p = np.array(ad.index.grids[0][var]) #assume only one grid! --> TODO: Extend to possible more grids
  p = np.squeeze(p) # squeeze the last axis from the array, because we have only 2 dimensions
  if MASK_DATA_WITH_VFRAC:
    vfrac = np.array(ad.index.grids[0]["vfrac"])
    vfrac = np.squeeze(vfrac)
    p = np.ma.masked_where( vfrac<=vfrac_cutoff, p)
  return p, current_time

def PrintProgress(i, plotfiles):
  ny = len(plotfiles)
  progress = int(100.0 * float(i) / (ny - 1))
  print('[{:3d}%] Reading plotfile {}'.format(progress, plotfiles[i]))

yt.funcs.mylog.setLevel(50)
output_dir = dataPath+'/Plotfiles/Plenum%i'%(plenum_number)

dirs = os.listdir(output_dir)
plotfiles = ['{}/{}'.format(output_dir, plt) for plt in dirs]
list.sort(plotfiles)
plotfiles = plotfiles[:]
# print(plotfiles)

ds = yt.load(plotfiles[0])

valueStrings = [el[1] for el in ds.field_list]
print(valueStrings)
if valueString not in valueStrings:
  raise ValueError('Wrong valueString, available valueStrings are:\n'+", ".join(valueStrings) )

ad = ds.all_data()
LeftEgdes = ad.index.grids[0].LeftEdge
RightEdges = ad.index.grids[-1].RightEdge

p, current_time = loadData(plotfiles[0], valueString)
PrintProgress(0, plotfiles)

nx = p.shape
ny = len(plotfiles) # how many plotfiles do we have


ts = np.zeros((ny,1))
ts[0] = ds.current_time
if MASK_DATA_WITH_VFRAC:
  imdata = np.ma.zeros(shape=(*nx, ny))
else:
  imdata = np.ma.zeros(shape=(*nx, ny))
imdata[:,:,0] = p

output_path = dataPath+"Plotfiles/Plenum%i_%s/"%(plenum_number, valueString)
if not os.path.exists(output_path):
  os.makedirs(output_path)

def plotImshow(plotfile, valueString, imdata, current_time, vmin, vmax):
  
  fig = plt.figure(figsize=(10,10))
  im = plt.imshow(imdata.T, interpolation='none', extent=(LeftEgdes[0], RightEdges[0], LeftEgdes[1], RightEdges[1]), aspect='auto', vmin=vmin, vmax=vmax)
  cbar = plt.colorbar(im)
  cbar.set_label(valueString)
  plt.title("time = %2.6f"%current_time)
  fig.savefig(output_path+valueString+"_"+plotfile.split("/")[-1]+".png" )
  fig.clf()
  plt.close(fig)


for i in range(1,ny):
   PrintProgress(i, plotfiles)
   p, current_time = loadData(plotfiles[i], valueString)
   ts[i] = current_time
   imdata[:,:,i] = p

# min and max values to adjust all colorbars
vmin = np.min(imdata)
vmax = np.max(imdata)
print(vmin, vmax)
for i in range(0, ny):
   plotImshow(plotfiles[i], valueString, imdata[:,:,i], ts[i], vmin, vmax)

['Density', 'Energy', 'Momentum_0', 'Momentum_1', 'Pressure', 'Species_0', 'Species_1', 'SpeedOfSound', 'vfrac']
[  0%] Reading plotfile /home/christian/FVS_develop/FiniteVolumeSolver/build_Release_2d_EB/SEC_Plenum//Plotfiles/Plenum0/plt000000000
[  0%] Reading plotfile /home/christian/FVS_develop/FiniteVolumeSolver/build_Release_2d_EB/SEC_Plenum//Plotfiles/Plenum0/plt000000008
0.4290603535238548 1.0000000000000002
